In [12]:
!pip install folium # To visualization
!pip install geocoder # To get my current location
!pip install pandas

  Using cached pandas-1.3.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.5 MB)


#위치 설정

In [2]:
# My current location
import geocoder
myloc = geocoder.ip('me').latlng
print(myloc)

[36.3491, 127.3849]


In [3]:
import urllib
import json

# Naver API Settings
CLIENT_ID = "irb5bjmqxk"
CLIENT_SECRET = "I9d6R41EXBoOvOc8HrOVb8bhngvNPGE3ZXKxEjOB"
GEOCODING_URL = "https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode"

# Search XY location of the address from Naver Geocoding API
def geocoding(query):
  client_id = CLIENT_ID
  client_secret = CLIENT_SECRET
  encText = urllib.parse.quote(query)
  url = f"{GEOCODING_URL}?query={encText}" # Naver Geocoding URL
  request = urllib.request.Request(url)
  request.add_header("X-NCP-APIGW-API-KEY-ID", client_id)
  request.add_header("X-NCP-APIGW-API-KEY", client_secret)
  response = urllib.request.urlopen(request)
  rescode = response.getcode()
  if (rescode == 200):
    response_body = response.read()
    result = response_body.decode('utf-8')
    result = json.loads(result)
    return result
  else:
    print("Error Code: " + rescode)
    return null

def getloc_from_geocoding(query):
  g = geocoding(query)['addresses'][0]
  location = [float(g['y']), float(g['x'])]
  return location

#데이터 로딩

In [78]:
import pandas as pd
DATADIR = "./data"
LOCATIONS = {
    "Yuseonggu": getloc_from_geocoding("대전광역시 유성구"),
    "Donggu": getloc_from_geocoding("대전광역시 동구"),
}
PARKING_LOTS_FILE = {
    "Yuseonggu": f"{DATADIR}/대전광역시_유성구_주차장정보_20200316_1594707282283_23187.csv",
    "Donggu": f"{DATADIR}/대전광역시_동구_주차장정보_20210430_1621994509493_21712.csv",
}

# Data Locations
class Region:
    def __init__(self, name):
        self.name = name
        
    def get_parking_lots(self):
        file = PARKING_LOTS_FILE[self.name]
        return pd.read_csv(file, encoding = "utf-8")
    
    def get_location(self):
        return LOCATIONS[self.name]
    
def getloc(df):
    return df[['위도', '경도']]

In [79]:
# Region Instances
yuseonggu = Region("Yuseonggu")
donggu = Region("Donggu")

print(f"유성구: {yuseonggu.get_location()}")
print(f"동구: {donggu.get_location()}")

유성구: [36.3623219, 127.3562683]
동구: [36.312169, 127.454884]


#지도 시각화

In [83]:
import folium
myloc = yuseonggu.get_location()

In [84]:
# Folium map generation
m = folium.Map(location = myloc,
               zoom_start = 13,
               )
m

In [85]:
m = folium.Map(location = myloc,
               zoom_start = 13,
               tiles = "Stamen Terrain",
               )

# Placing Spot Markers
folium.Marker(
    location = myloc,
    #popup = "Current Location",
    tooltip = "Current Location",
    icon = folium.Icon(color = "blue"),
).add_to(m)

# Click and popup locations
m.add_child(folium.LatLngPopup())

m

In [86]:
m = folium.Map(
    location = myloc,
    tiles = "Stamen Toner",
    zoom_start = 15,
)

# Placing Area Circules
folium.Circle(
    location = myloc,
    radius = 500, # meters
    #popup = "Popup String",
    tooltip = "My Region",
    color = "crimson",
    fill = True,
    fill_color = "crimson",
).add_to(m)

m

#유성구, 동구 주차장 오버레이

In [96]:
m = folium.Map(
    location = yuseonggu.get_location(),
    zoom_start = 12,
)

parking_df = getloc(yuseonggu.get_parking_lots())
parking_df.dropna(inplace = True)
parking_df.reset_index(drop = True, inplace = True)
for i in range(len(parking_df)):
    loc = [parking_df['위도'][i], parking_df['경도'][i]]
    folium.Circle(
        location = loc,
        rarius = 10,
        fill = True,
        fill_color = "blue",
    ).add_to(m)

m

In [97]:
m = folium.Map(
    location = donggu.get_location(),
    zoom_start = 12,
)

parking_df = getloc(donggu.get_parking_lots())
parking_df.dropna(inplace = True)
parking_df.reset_index(drop = True, inplace = True)
for i in range(len(parking_df)):
    loc = [parking_df['위도'][i], parking_df['경도'][i]]
    folium.Circle(
        location = loc,
        rarius = 10,
        color = "red",
        fill = True,
        fill_color = "red",
    ).add_to(m)

m